In [2]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import scipy.stats
from prettytable import PrettyTable
%matplotlib inline

You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [76]:
dg = pd.read_csv('LRM_1_hour.csv')

In [9]:
len(dg.stool_day_total)

978682

In [78]:
def logistic(X_train, X_test, y_train):
    model = LogisticRegression()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test) 
    return y_pred

In [79]:
cols = ['pulserate_SE',
       'pulserate_DFA', 'pulserate_ADF', 'pulserate_Mean', 'pulserate_Var',
       'ecg_resprate_SE', 'ecg_resprate_DFA', 'ecg_resprate_ADF',
       'ecg_resprate_Mean', 'ecg_resprate_Var', 'spo2_SE', 'spo2_DFA',
       'spo2_ADF', 'spo2_Mean', 'spo2_Var', 'heartrate_SE', 'heartrate_DFA',
       'heartrate_ADF', 'heartrate_Mean', 'heartrate_Var', 'peep_SE',
       'peep_DFA', 'peep_ADF', 'peep_Mean', 'peep_Var', 'pip_SE', 'pip_DFA',
       'pip_ADF', 'pip_Mean', 'pip_Var', 'map_SE', 'map_DFA', 'map_ADF',
       'map_Mean', 'map_Var', 'tidalvol_SE', 'tidalvol_DFA', 'tidalvol_ADF',
       'tidalvol_Mean', 'tidalvol_Var', 'minvol_SE', 'minvol_DFA',
       'minvol_ADF', 'minvol_Mean', 'minvol_Var', 'ti_SE', 'ti_DFA', 'ti_ADF',
       'ti_Mean', 'ti_Var', 'fio2_SE', 'fio2_DFA', 'fio2_ADF', 'fio2_Mean',
       'fio2_Var', 'abdomen_girth', 'urine', 'total_intake',
       'totalparenteralvolume', 'new_ph', 'gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status',
        'baby_type', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'rbs', 'temp',
       'currentdateweight', 'currentdateheight',  'gestation',
        'tpn-tfl', 'abd_difference', 'stool_day_total',
       'typevalue_Antibiotics', 'typevalue_Inotropes','mean_bp', 'sys_bp',
       'dia_bp']

In [86]:
def training(X,y):
    auc_roc = []
    training = []

    for i in range(25):
        try:
            X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=0.3)
            # SMOTE
            sm = SMOTE(k_neighbors = 2)
            X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)
            y_smote = logistic(X_train_sm, X_test, y_train_sm)
            y_smote_1 = logistic(X_train_sm,X_train,y_train_sm)
            auc_roc.append(roc_auc_score(y_test,y_smote))
            training.append(roc_auc_score(y_train,y_smote_1))
            continue
        except Exception as e:
            continue
    return auc_roc,y_test,y_smote


            


In [80]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)

In [87]:
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_a = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_1 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [88]:
print(an)



([0.3, 0.75, 0.44999999999999996, 0.25, 0.5, 0.55, 0.44999999999999996, 0.4583333333333334, 0.6666666666666666, 0.3333333333333333, 0.55, 0.6666666666666667, 0.3, 0.41666666666666663, 0.75, 0.44999999999999996, 0.2, 0.25, 0.65, 0.6666666666666667, 0.3, 0.55, 0.5, 0.6666666666666667], 3     1
0     1
8     1
15    0
6     0
21    1
2     0
Name: dischargestatus, dtype: int64, array([1, 1, 1, 1, 0, 1, 1]))


In [16]:
dg

,Unnamed: 0,uhid,dischargestatus,pulserate_SE,pulserate_DFA,pulserate_ADF,pulserate_Mean,pulserate_Var,ecg_resprate_SE,ecg_resprate_DFA,...,currentdateheight,tpn-tfl,mean_bp,sys_bp,dia_bp,abd_difference,stool_day_total,total_intake,typevalue_Antibiotics,typevalue_Inotropes
0,0,RNEH.0000013713,1,NaN,NaN,NaN,-999.000000,0.000000,NaN,NaN,...,-999.000000,-659304.093966,-999.000000,-999.000000,-999.000000,NaN,1.000000,-647386.80,-719280.0,-719280.0
1,1,RNEH.0000013396,0,1.222964,0.947649,-5.400535,144.180556,112.556289,NaN,NaN,...,-999.000000,-701281.196809,-974.506944,-974.183333,-974.601389,NaN,0.000000,-699673.68,-719280.0,-719280.0
2,2,RSHI.0000015178,1,0.000535,1.827419,-2.572479,-17.683333,146436.121944,0.000470,1.669744,...,-999.000000,-659280.000000,-898.647222,-892.770833,-895.159722,NaN,1.000000,-654018.00,-659280.0,-659340.0
3,3,RSHI.0000023451,1,0.005463,1.714992,-1.254377,-337.816667,314077.144167,0.002265,1.704811,...,-999.000000,-658283.641791,-939.530556,-938.447222,-939.927778,NaN,1.894444,-652210.50,-659280.0,-719280.0
4,4,RSHI.0000016373,1,NaN,NaN,NaN,-999.000000,0.000000,NaN,NaN,...,40.500000,-719280.000000,-999.000000,-999.000000,-999.000000,NaN,0.000000,-650268.00,-719280.0,-719280.0
5,5,RSHI.0000017471,1,0.644101,1.187243,-7.161844,139.240278,103.246433,1.948362,0.807091,...,-999.000000,-719280.000000,-973.283333,-972.791667,-973.491667,NaN,0.000000,-701447.00,-683280.0,-683316.0
6,6,RSHI.0000023452,0,0.502928,1.185560,-1.965723,137.076389,64.926109,1.808927,0.822991,...,-999.000000,-658283.605911,-899.008333,-878.643056,-881.381944,NaN,0.963889,-652149.50,-659280.0,-719280.0
7,7,RSHI.0000021953,1,1.061370,1.156357,-10.875079,130.019444,128.919066,0.005038,1.619452,...,-872.561111,-691280.000000,-930.940278,-920.972222,-923.172222,NaN,4.000000,-688816.00,-719280.0,-719280.0
8,8,RSHI.0000019707,1,0.000786,1.878452,-4.225700,75.941667,66226.871597,0.000160,1.624085,...,-999.000000,-719280.000000,-908.212500,-906.747222,-908.815278,NaN,1.527778,-719280.00,-719280.0,-719280.0
9,9,RNEH.0000013016,0,0.106244,1.257930,-5.537551,129.376389,9123.070831,NaN,NaN,...,-999.000000,-659293.975155,-967.280556,-966.752778,-967.408333,NaN,0.433333,-653157.60,-719280.0,-719280.0


In [89]:
dg = pd.read_csv('LRM_6_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_6 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_6 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_6 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [90]:
dg = pd.read_csv('LRM_12_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_12 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_12 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_12 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [91]:
dg = pd.read_csv('LRM_48_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_48 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_48 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_48 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [30]:
dg = pd.read_csv('LRM_1_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_1w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_1w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [92]:
dg = pd.read_csv('LRM_2_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_2w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_2w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_2w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [93]:
dg = pd.read_csv('LRM_3_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_3w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_3w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_3w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [94]:
dg = pd.read_csv('LRM_4_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_4w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_4w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_4w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [95]:
l = [["1 Hour" ,c_a, ppv_1,npv_1],["6 Hour" ,c_6, ppv_6,npv_6],["12 Hour" ,c_12, ppv_12,npv_12],["48 Hour" ,c_48, ppv_48,npv_48],["1 Week" ,c_1w, ppv_1w,npv_1w],["2 Week" ,c_2w, ppv_2w,npv_3w],["3 Week" ,c_3w, ppv_3w,npv_3w],["4 Week" ,c_4w, ppv_4w,npv_4w]]

table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','NPV'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+-----------------------+------+------+
| Parameter |        AUC-ROC        | PPV  | NPV  |
+-----------+-----------------------+------+------+
|   1 Hour  | (0.484, 0.414, 0.555) | 0.33 | 1.0  |
|   6 Hour  | (0.454, 0.371, 0.537) | 0.0  | 0.33 |
|  12 Hour  |  (0.471, 0.401, 0.54) | 0.67 | 0.5  |
|  48 Hour  |  (0.71, 0.614, 0.805) | 0.5  | 0.4  |
|   1 Week  | (0.478, 0.421, 0.536) | 0.0  | 0.75 |
|   2 Week  | (0.661, 0.582, 0.739) | 0.0  | 0.5  |
|   3 Week  | (0.619, 0.539, 0.698) | 1.0  | 0.5  |
|   4 Week  | (0.645, 0.561, 0.729) | 0.25 | 0.67 |
+-----------+-----------------------+------+------+
